In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
search_url = "https://fbref.com/en/comps/1/World-Cup-Stats"
response = requests.get(search_url)
soup = BeautifulSoup(response.content)

In [3]:
all_teams_table = soup.find('div', {'id': 'div_results202210_overall'}).find('table')
team_ids = {}
for row in all_teams_table.find_all('tr'):
    curr_row_data = row.find('td' , {'data-stat': 'team'})
    if curr_row_data and curr_row_data.a:
        team_name = curr_row_data.a.text
        team_id = curr_row_data.a.attrs['href'].split('/')[3]
        team_ids[team_name] = team_id

In [4]:
team_ids

{'Argentina': 'f9fddd6e',
 'France': 'b1b36dcd',
 'Croatia': '7b08e376',
 'Morocco': 'af41ccda',
 'Netherlands': '5bb5024a',
 'England': '1862c019',
 'Brazil': '304635c3',
 'Portugal': '4a1b4ea8',
 'Japan': 'ffcf1690',
 'Senegal': '9ab5c684',
 'Australia': 'b90bf4f9',
 'Switzerland': '81021a70',
 'Spain': 'b561dd30',
 'United States': '0f66725b',
 'Poland': '8912dcf0',
 'Korea Republic': '473f0fbf',
 'Germany': 'c1e40422',
 'Ecuador': '123acaf8',
 'Cameroon': '896550da',
 'Uruguay': '870e020f',
 'Tunisia': 'a7c7562a',
 'Mexico': 'b009a548',
 'Belgium': '361422b9',
 'Ghana': '9349828d',
 'Saudi Arabia': '6e84edac',
 'IR Iran': '6a08f71e',
 'Costa Rica': '1ea5ab66',
 'Denmark': '29a4e4af',
 'Serbia': '1d6f5c9b',
 'Wales': 'b1bbcad3',
 'Canada': '9c6d90a0',
 'Qatar': '9b696ed1'}

In [28]:
def get_player_match_data_by_team(team_id, team_name):
    url = f"https://fbref.com/en/squads/{team_id}/2022/matchlogs/c1/schedule/{team_name}-Men-Scores-and-Fixtures-World-Cup"
    soup = BeautifulSoup(requests.get(url).content)
    table = soup.find('table' , {'class': ['stats_table', 'sortable','min_width', 'now_sortable sticky_table', 'eq1', 're1' ,'le1']})
    table_rows = [table_row for table_row in table.tbody.find_all('tr')]
    matches = []
    for row in table_rows:
        if [td for td in row][-2].text != '':
            matches.append([td for td in row][-2].a.attrs['href'])

    players = []
    number_of_matches = 1
    
    for match in matches:
        match_link = "https://fbref.com" + match
        match_soup = BeautifulSoup(requests.get(match_link).content)
        first_team_name, second_team_name = match_soup.find('h1').text.split("Match Report")[0].split('vs.')
        match_table = match_soup.find('table', id = f"stats_{team_id}_summary")
        match_table_rows = [table_row for table_row in match_table.tbody.find_all('tr')]
        misc_stats_table = match_soup.find('table', id = f"stats_{team_id}_misc")
        own_goals = [int(td.text) for td in misc_stats_table.find_all('td') if td.attrs['data-stat'] == 'own_goals']
        
        i = 0
        for row in match_table_rows:
            player_dict = {
                "team": team_name ,
                "first_team_name":first_team_name, 
                "second_team_name": second_team_name,
                "own_goals": own_goals[i]
            }
            for td in row:
                player_dict[td.attrs['data-stat']] = td.text
            
            players.append(player_dict)
            i += 1

        print(f" {team_name} : Match {number_of_matches} Done")
        number_of_matches += 1
        time.sleep(30)
    
    return players

In [29]:
all_player_match_data = []

for team_name, team_id in team_ids.items():
    all_player_match_data.append(get_player_match_data_by_team(team_id, team_name))

 Argentina : Match 1 Done
 Argentina : Match 2 Done
 Argentina : Match 3 Done
 Argentina : Match 4 Done
 Argentina : Match 5 Done
 Argentina : Match 6 Done
 Argentina : Match 7 Done
 France : Match 1 Done
 France : Match 2 Done
 France : Match 3 Done
 France : Match 4 Done
 France : Match 5 Done
 France : Match 6 Done
 France : Match 7 Done
 Croatia : Match 1 Done
 Croatia : Match 2 Done
 Croatia : Match 3 Done
 Croatia : Match 4 Done
 Croatia : Match 5 Done
 Croatia : Match 6 Done
 Croatia : Match 7 Done
 Morocco : Match 1 Done
 Morocco : Match 2 Done
 Morocco : Match 3 Done
 Morocco : Match 4 Done
 Morocco : Match 5 Done
 Morocco : Match 6 Done
 Morocco : Match 7 Done
 Netherlands : Match 1 Done
 Netherlands : Match 2 Done
 Netherlands : Match 3 Done
 Netherlands : Match 4 Done
 Netherlands : Match 5 Done
 England : Match 1 Done
 England : Match 2 Done
 England : Match 3 Done
 England : Match 4 Done
 England : Match 5 Done
 Brazil : Match 1 Done
 Brazil : Match 2 Done
 Brazil : Match

In [30]:
all_player_match_data_flattened = [player for team in all_player_match_data for player in team]

In [31]:
player_match_df = pd.DataFrame(all_player_match_data_flattened)

In [ ]:
player_match_df.to_csv('../data/player_match.csv', index = False)